# CARA-Metabolomics LC-MS/MS Annotation analysis - C18 pos

**Author**: Louis-Felix Nothias, Feb 2021

**Objective**: 
- Explore the annotations with feature-based molecular networking (FBMN)
- Check if SIRIUS annotations are consistent with spectral library matches at the molecular formula (MF) and class level. We can also check with for analogue library match.

**Additional ideas**

In [24]:
import pandas as pd     
import numpy as np
import altair as alt
from get_stats_annotation import *
from check_annotation import *
from visualize_annotation import *
pd.set_option('mode.chained_assignment', None)

### Prepare input annotation files

In [28]:
# Importing the feature metadata for CMN and FBMN
FBMN = pd.read_csv('input/FBMN/C18pos_feature_metadata.tsv',  sep='\t', index_col=0, header=0, low_memory=False)

### Lets look at the annotations metadata per tools

The annotation metadata originating from a specific share the same prefix:

**GNPS tools**:

- **Molecular networking** (column prefix: `'GNPS_'`).
- **Spectral library search** (column prefix: `'GNPS_LIB_'`).
- **Spectral library search in analogue mode** (column prefix: `'GNPS_LIBA_'`).
- **Passatutto FDR controlled spectral lib match** (column prefix: `'PASSA_FDR_'`).

**SIRIUS tools**:

- **ZODIAC**: Molecular formula annotation (column prefix: `'SIR_MF_Zod'`).
- **CSI:FingerID**: Putative structure annotation (column prefix: `'CSI_'`).
- **CANOPUS**: Putative chemical class annotation (column prefix: `'CAN_'`).


#### We can limit to a subset of features

In [29]:
features_of_interest_p_value = pd.read_csv('Input/FBMN/statistical_analysis_results/C18_pos.p_value.tsv',  sep='\t', index_col=0, header=0)

print('Features to filter: '+ str(features_of_interest.shape))
print('Features in the table before: '+ str(FBMN.shape[0]))
FBMN  = FBMN[FBMN.index.isin(features_of_interest_p_value.index)]
print('Features in the table after p value filter: '+ str(FBMN.shape[0]))

Features to filter: (11981, 14)
Features in the table before: 15131
Features in the table after p value filter: 11981


In [30]:
# This prints the metadata columns per annotations using the respective prefix.
def show_metadata_tools(table, metadata_prefix):
    metadata = []
    for x in table.columns:
        if x.startswith(metadata_prefix):
            metadata.append(str(x))
    print(metadata)

In [31]:
show_metadata_tools(FBMN,'GNPS_LIB_')

['GNPS_LIB_SpectrumID', 'GNPS_LIB_Compound_Name', 'GNPS_LIB_Ion_Source', 'GNPS_LIB_Instrument', 'GNPS_LIB_Compound_Source', 'GNPS_LIB_PI', 'GNPS_LIB_Data_Collector', 'GNPS_LIB_Adduct', 'GNPS_LIB_Precursor_MZ', 'GNPS_LIB_ExactMass', 'GNPS_LIB_Charge', 'GNPS_LIB_CAS_Number', 'GNPS_LIB_Pubmed_ID', 'GNPS_LIB_Smiles', 'GNPS_LIB_INCHI', 'GNPS_LIB_INCHI_AUX', 'GNPS_LIB_Library_Class', 'GNPS_LIB_IonMode', 'GNPS_LIB_UpdateWorkflowName', 'GNPS_LIB_LibraryQualityString', 'GNPS_LIB_SpectrumFile', 'GNPS_LIB_LibraryName', 'GNPS_LIB_MQScore', 'GNPS_LIB_Organism', 'GNPS_LIB_TIC_Query', 'GNPS_LIB_RT_Query', 'GNPS_LIB_MZErrorPPM', 'GNPS_LIB_SharedPeaks', 'GNPS_LIB_MassDiff', 'GNPS_LIB_LibMZ', 'GNPS_LIB_SpecMZ', 'GNPS_LIB_SpecCharge', 'GNPS_LIB_FileScanUniqueID', 'GNPS_LIB_NumberHits', 'GNPS_LIB_tags', 'GNPS_LIB_MoleculeExplorerDatasets', 'GNPS_LIB_MoleculeExplorerFiles', 'GNPS_LIB_InChIKey', 'GNPS_LIB_InChIKey-Planar', 'GNPS_LIB_superclass', 'GNPS_LIB_class', 'GNPS_LIB_subclass']


# Feature-Based Molecular Networking
## General annotation statistics

In [32]:
zodiac_score_thresh=0.7
ionisation_mode='pos'
ppm_error=25

In [33]:
get_stats_annotation(FBMN, zodiac_score_thresh, ionisation_mode, ppm_error)

Features = 11981
 
==== GNPS =====
In networks = 3519
Number of networks = 848
Valid library annotations = 298
Library annotations in analogue mode= 1155
PASSATUTTO FDR-controlled library annotations = 168
PASSATUTTO FDR-controlled library annotations at 20% FDR = 155
PASSATUTTO FDR-controlled library annotations at 10% FDR = 18
 
==== SIRIUS =====
Features with SIRIUS annotation = 5680
SIRIUS ZODIAC MF with ZodiacScore > 0.7 = 3841
CSIFingerID annotations = 1879
CANOPUS annotations = 3064
CANOPUS annotations at the subclass level= 2971
CANOPUS annotations at the subclass level= 2420
CANOPUS annotations at the level 5 = 1406
 
==== General annotation statistics =====
Number of features = 11981
Annotated features = 6203
Annotated features or in network = 8017
Single nodes = 8462
Single nodes and unnnannotated = 4166
 


### View all annotations

In [34]:
get_stats_annotation.final_table_rel
get_stats_annotation.final_table

,Annotation tool,Count
0,Features,11981
1,GNPS - in networks,3519
2,GNPS - lib. match,298
3,GNPS - lib. match analogue,1155
4,PASSATUTTO FDR 20%,155
5,PASSATUTTO FDR 10%,18
6,SIRIUS - Annotated features,5680
7,SIRIUS - MF with ZodScore >0.7,3841
8,SIRIUS - structure,1879
9,SIRIUS - chemical class,3064


In [35]:
make_barchart(get_stats_annotation.final_table) # Absolute count
make_barchart_rel(get_stats_annotation.final_table_rel)

alt.Chart(...)

### View GNPS annotations

In [36]:
make_barchart(get_stats_annotation.table_gnps) # Absolute count
make_barchart_rel(get_stats_annotation.table_gnps_rel)

alt.Chart(...)

### View SIRIUS annotations

In [37]:
make_barchart(get_stats_annotation.table_sirius) # Absolute count
make_barchart_rel(get_stats_annotation.table_sirius_rel)

alt.Chart(...)

# Look at GNPS/SIRIUS annotation consistency

In [38]:
check_matching_annotations(FBMN, zodiac_score_thresh, ionisation_mode='pos', library_mode = 'reg', canopus_level= 'spec', 
                           cosine=0.7, shared_peaks=6, ppm_error=25)

=== Looking at match between GNPS library in REGULAR mode and SIRIUS annotation ===
Usable GNPS/SIRIUS annotations = 51
Usable GNPS/SIRIUS annot. w. ZodiacScore > 0.7 = 51
Check with CANOPUS SPECIFIC classification levels
 
====== Match for molecular formulas =======
MF match = 29
MF match score = 28
 
====== Match between GNPS lib superclass/class/subclass and SIRIUS CANOPUS level(s) =======
Classified pairs considered = 44
Superclass annotation pairs = 37
Superclass match all = 37, 0.84%
Class annotation pairs = 44
Class match = 35, 0.80%
Subclass annotation pairs = 40
Subclass match all = 25, 0.62%


### Molecular formula annotation consistency between GNPS/SIRIUS

In [39]:
check_matching_annotations.table_matching

,Matching level,Count,Relative
0,Usable MF pairs,51,1.16
1,Usable MF pairs w. ZodiacScore>0.7,44,1.00
2,Matching molecular formula,29,0.66
3,Matching molecular w. ZodiacScore>0.7,28,0.64


In [40]:
make_barchart_match(check_matching_annotations.table_matching) ### This one for absolute values
make_barchart_match_rel(check_matching_annotations.table_matching)

alt.Chart(...)

### Chemical class annotation consistency between GNPS-SIRIUS/CANOPUS

In [41]:
check_matching_annotations.table_class_matching

,Matching level,Count,Relative
0,Available pairs,51,1.16
1,Classified pairs w. ZodiacScore>0.7,44,1.00
2,Matching superclass,37,0.84
3,Matching class,35,0.80
4,Matching subclass,25,0.57


In [42]:
check_matching_annotations.table_class_matching
make_barchart_match(check_matching_annotations.table_class_matching) ### This one for absolute values
make_barchart_match_rel(check_matching_annotations.table_class_matching)

alt.Chart(...)

# Additional, detailed views (Optional) 
## Below are to view distribution of correct/incorrect annotations

### View Molecular Formula (only for REGULAR library search)

In [43]:
dist_plot(check_matching_annotations.MF_pairs,'MF_match', zodiac_score_thresh)

alt.VConcatChart(...)

### View classification results

In [44]:
# Superclass level
#dist_plot(check_matching_annotations.superclass_match_all_total,'Match_GNPSsuperclass-SIRIUS',zodiac_score_thresh)
dist_plot(check_matching_annotations.class_match_all_total,'Match_GNPSclass-SIRIUS',zodiac_score_thresh)
#dist_plot(check_matching_annotations.subclass_match_all_total,'Match_GNPSsubclass-SIRIUS',zodiac_score_thresh)

alt.VConcatChart(...)

### View details of incorrect MF annotations (only for REGULAR library search)

In [45]:
# This is used to display the entire table in the notebook
from IPython.display import display, HTML
show_non_matching_MF = check_matching_annotations.MF_no_match[['SIR_MF_Zod_ZodiacScore','GNPS_LIB_INCHI_MF','SIR_MF_Zod_molecularFormula',
                                                               'GNPS_LIB_Adduct','SIR_MF_Zod_adduct',
                                                               'GNPS_LIB_superclass','CAN_superclass',
                                                               'GNPS_LIB_class','CAN_class',
                                                               'GNPS_LIB_MQScore', 'GNPS_LIB_MZErrorPPM', 'GNPS_LIB_SharedPeaks', 'GNPS_LIB_SpecCharge','GNPS_LIB_SpecMZ']]
show_non_matching_MF.sort_values(['GNPS_LIB_SpecMZ','SIR_MF_Zod_ZodiacScore'], inplace=True, ascending = (False, False))

display(HTML(show_non_matching_MF.to_html()))

,SIR_MF_Zod_ZodiacScore,GNPS_LIB_INCHI_MF,SIR_MF_Zod_molecularFormula,GNPS_LIB_Adduct,SIR_MF_Zod_adduct,GNPS_LIB_superclass,CAN_superclass,GNPS_LIB_class,CAN_class,GNPS_LIB_MQScore,GNPS_LIB_MZErrorPPM,GNPS_LIB_SharedPeaks,GNPS_LIB_SpecCharge,GNPS_LIB_SpecMZ
#featureID,,,,,,,,,,,,,,
6114,0.946,C37H70O5,C35H72O5,M+H,[M + Na]+,Lipids and lipid-like molecules,Lipids and lipid-like molecules,Glycerolipids,Glycerolipids,0.735448,2.152270,7.0,1.0,595.531
35135,0.458,C22H30Cl2N10,C23H24N10O4,M+H,[M + H]+,Benzenoids,Organic acids and derivatives,Benzene and substituted derivatives,Carboxylic acids and derivatives,0.923418,4.953260,10.0,1.0,505.208
35121,0.567,C22H30Cl2N10,C24H36Cl2N4O2,M+H,[M + Na]+,Benzenoids,Benzenoids,Benzene and substituted derivatives,Benzene and substituted derivatives,0.847934,6.342600,12.0,1.0,505.207
2691,0.870,C28H46O4,C26H48O4,M+H,[M + Na]+,Benzenoids,Organic acids and derivatives,Benzene and substituted derivatives,Carboxylic acids and derivatives,0.809313,4.911770,8.0,1.0,447.345
4766,0.680,C30H50O,C28H50,M+H-H2O,[M + Na]+,Lipids and lipid-like molecules,Lipids and lipid-like molecules,Prenol lipids,Prenol lipids,0.751144,9.318160,12.0,1.0,409.379
27030,1.000,C21H30O5,C19H32O5,M+H,[M + Na]+,Lipids and lipid-like molecules,Lipids and lipid-like molecules,Steroids and steroid derivatives,Steroids and steroid derivatives,0.775606,5.293280,11.0,1.0,363.215
27189,0.934,C12H22O11,C14H20BFN4O4,M+NH4,[M + Na]+,Lipids and lipid-like molecules,Benzenoids,Fatty Acyls,Benzene and substituted derivatives,0.808862,3.897840,8.0,1.0,360.149
33405,0.993,C21H40O4,C19H40O3,M+H-H2O,[M + Na]+,Lipids and lipid-like molecules,Lipids and lipid-like molecules,Glycerolipids,Glycerolipids,0.864691,6.745920,11.0,1.0,339.287
9081,0.634,C18H15O4P,C20H16O2,M+H,[M + K]+,Organic acids and derivatives,Benzenoids,Organic phosphoric acids and derivatives,Benzene and substituted derivatives,0.856392,6.624560,10.0,1.0,327.076


### View details of correct MF annotations (only for REGULAR library search)

In [46]:
# This is used to display the entire table in the notebook
from IPython.display import display, HTML
show_matching_MF = check_matching_annotations.MF_match[['SIR_MF_Zod_ZodiacScore','GNPS_LIB_INCHI_MF','SIR_MF_Zod_molecularFormula',
                                                               'GNPS_LIB_Adduct','SIR_MF_Zod_adduct',
                                                               'GNPS_LIB_superclass','CAN_superclass',
                                                               'GNPS_LIB_class','CAN_class',
                                                               'GNPS_LIB_MQScore', 'GNPS_LIB_MZErrorPPM', 'GNPS_LIB_SharedPeaks', 'GNPS_LIB_SpecCharge','GNPS_LIB_SpecMZ']]
show_matching_MF.sort_values(['GNPS_LIB_SpecMZ','SIR_MF_Zod_ZodiacScore'], inplace=True, ascending = (False, False))

display(HTML(show_matching_MF.to_html()))

,SIR_MF_Zod_ZodiacScore,GNPS_LIB_INCHI_MF,SIR_MF_Zod_molecularFormula,GNPS_LIB_Adduct,SIR_MF_Zod_adduct,GNPS_LIB_superclass,CAN_superclass,GNPS_LIB_class,CAN_class,GNPS_LIB_MQScore,GNPS_LIB_MZErrorPPM,GNPS_LIB_SharedPeaks,GNPS_LIB_SpecCharge,GNPS_LIB_SpecMZ
#featureID,,,,,,,,,,,,,,
3846,0.974,C37H70O5,C37H70O5,M+H-H2O,[M - H2O + H]+,Lipids and lipid-like molecules,Lipids and lipid-like molecules,Glycerolipids,Glycerolipids,0.765873,2.64213,12.0,1.0,577.521
24697,0.999,C35H68O5,C35H68O5,M+H-H2O,[M - H2O + H]+,Lipids and lipid-like molecules,Lipids and lipid-like molecules,Glycerolipids,Glycerolipids,0.863114,1.88140,12.0,1.0,551.502
769,0.975,C31H52O3,C31H52O3,[M+NH4]+,[M + H3N + H]+,Lipids and lipid-like molecules,Lipids and lipid-like molecules,Prenol lipids,Sphingolipids,0.824130,1.80458,8.0,1.0,490.424
9160,0.863,C26H42O4,C26H42O4,M+H,[M + H]+,Benzenoids,Benzenoids,Benzene and substituted derivatives,Benzene and substituted derivatives,0.777877,5.02178,7.0,1.0,419.314
757,0.978,C24H27NO2,C24H27NO2,[M+H]+,[M + H]+,Benzenoids,Benzenoids,Benzene and substituted derivatives,Benzene and substituted derivatives,0.921233,2.78037,8.0,1.0,362.210
316,1.000,C21H40O4,C21H40O4,M+H,[M + H]+,Lipids and lipid-like molecules,Lipids and lipid-like molecules,Glycerolipids,Glycerolipids,0.795997,4.44141,9.0,1.0,357.298
10362,0.539,C20H37NO3,C20H37NO3,M+H,[M + H]+,Organic acids and derivatives,Lipids and lipid-like molecules,Carboxylic acids and derivatives,Fatty Acyls,0.791505,5.91905,8.0,1.0,340.283
257,1.000,C19H36O4,C19H36O4,M+H,[M + H]+,Lipids and lipid-like molecules,Lipids and lipid-like molecules,Glycerolipids,Glycerolipids,0.738777,3.89268,10.0,1.0,329.267
33935,1.000,C21H30O2,C21H30O2,M+H,[M + H]+,Organoheterocyclic compounds,Lipids and lipid-like molecules,Benzopyrans,Prenol lipids,0.792921,7.64798,8.0,1.0,315.229


### View details of incorrect/correct class annotations (valid only for REGULAR library search)

In [47]:
from IPython.display import display, HTML
show_matching_class = check_matching_annotations.class_match_all_total[[
                                                               'GNPS_LIB_superclass','CAN_superclass',
                                                               'GNPS_LIB_class','CAN_class',
                                                               'GNPS_LIB_subclass','CAN_subclass',
                                                               'SIR_MF_Zod_ZodiacScore','SIR_MF_Zod_molecularFormula',
                                                               'GNPS_LIB_Adduct','SIR_MF_Zod_adduct',
                                                               'GNPS_LIB_MQScore', 'GNPS_LIB_MZErrorPPM', 'GNPS_LIB_SharedPeaks',
                                                               'GNPS_LIB_SpecMZ','Match_GNPSsuperclass-SIRIUS',
                                                               'Match_GNPSclass-SIRIUS','Match_GNPSsubclass-SIRIUS']]
show_matching_class.sort_values(['Match_GNPSsuperclass-SIRIUS','Match_GNPSclass-SIRIUS','Match_GNPSsubclass-SIRIUS',
                                 'GNPS_LIB_SpecMZ','SIR_MF_Zod_ZodiacScore'], inplace=True, ascending = (False,False,False,False, False))

display(HTML(show_matching_class.to_html()))

,GNPS_LIB_superclass,CAN_superclass,GNPS_LIB_class,CAN_class,GNPS_LIB_subclass,CAN_subclass,SIR_MF_Zod_ZodiacScore,SIR_MF_Zod_molecularFormula,GNPS_LIB_Adduct,SIR_MF_Zod_adduct,GNPS_LIB_MQScore,GNPS_LIB_MZErrorPPM,GNPS_LIB_SharedPeaks,GNPS_LIB_SpecMZ,Match_GNPSsuperclass-SIRIUS,Match_GNPSclass-SIRIUS,Match_GNPSsubclass-SIRIUS
#featureID,,,,,,,,,,,,,,,,,
6114,Lipids and lipid-like molecules,Lipids and lipid-like molecules,Glycerolipids,Glycerolipids,Diradylglycerols,Diradylglycerols,0.946,C35H72O5,M+H,[M + Na]+,0.735448,2.152270,7.0,595.531,yes,yes,yes
3846,Lipids and lipid-like molecules,Lipids and lipid-like molecules,Glycerolipids,Glycerolipids,Diradylglycerols,Diradylglycerols,0.974,C37H70O5,M+H-H2O,[M - H2O + H]+,0.765873,2.642130,12.0,577.521,yes,yes,yes
24697,Lipids and lipid-like molecules,Lipids and lipid-like molecules,Glycerolipids,Glycerolipids,Diradylglycerols,Diradylglycerols,0.999,C35H68O5,M+H-H2O,[M - H2O + H]+,0.863114,1.881400,12.0,551.502,yes,yes,yes
9160,Benzenoids,Benzenoids,Benzene and substituted derivatives,Benzene and substituted derivatives,Benzoic acids and derivatives,Benzoic acids and derivatives,0.863,C26H42O4,M+H,[M + H]+,0.777877,5.021780,7.0,419.314,yes,yes,yes
27030,Lipids and lipid-like molecules,Lipids and lipid-like molecules,Steroids and steroid derivatives,Steroids and steroid derivatives,Hydroxysteroids,Hydroxysteroids,1.000,C19H32O5,M+H,[M + Na]+,0.775606,5.293280,11.0,363.215,yes,yes,yes
316,Lipids and lipid-like molecules,Lipids and lipid-like molecules,Glycerolipids,Glycerolipids,Monoradylglycerols,Monoradylglycerols,1.000,C21H40O4,M+H,[M + H]+,0.795997,4.441410,9.0,357.298,yes,yes,yes
33405,Lipids and lipid-like molecules,Lipids and lipid-like molecules,Glycerolipids,Glycerolipids,Monoradylglycerols,Monoradylglycerols,0.993,C19H40O3,M+H-H2O,[M + Na]+,0.864691,6.745920,11.0,339.287,yes,yes,yes
257,Lipids and lipid-like molecules,Lipids and lipid-like molecules,Glycerolipids,Glycerolipids,Monoradylglycerols,Monoradylglycerols,1.000,C19H36O4,M+H,[M + H]+,0.738777,3.892680,10.0,329.267,yes,yes,yes
1173,Lipids and lipid-like molecules,Lipids and lipid-like molecules,Glycerolipids,Glycerolipids,Monoradylglycerols,Monoradylglycerols,1.000,C17H34O4,M+H,[M + H]+,0.755989,5.534870,10.0,303.251,yes,yes,yes


### View details of correct/incorrect class annotations (valid only for ANALOGUE library search)

In [48]:
### View details of correct class annotations (only for REGULAR library search)
from IPython.display import display, HTML
show_matching_class = check_matching_annotations.class_match_all_total[[
                                                               'GNPS_LIBA_superclass','CAN_superclass',
                                                               'GNPS_LIBA_class','CAN_class',
                                                               'GNPS_LIBA_subclass','CAN_subclass',
                                                               'SIR_MF_Zod_ZodiacScore','SIR_MF_Zod_molecularFormula',
                                                               'GNPS_LIB_Adduct','SIR_MF_Zod_adduct',
                                                               'GNPS_LIB_MQScore', 'GNPS_LIB_MZErrorPPM', 'GNPS_LIB_SharedPeaks',
                                                               'GNPS_LIB_SpecMZ','Match_GNPSsuperclass-SIRIUS',
                                                               'Match_GNPSclass-SIRIUS','Match_GNPSsubclass-SIRIUS']]
show_matching_class.sort_values(['Match_GNPSsuperclass-SIRIUS','Match_GNPSclass-SIRIUS','Match_GNPSsubclass-SIRIUS',
                                 'GNPS_LIB_SpecMZ','SIR_MF_Zod_ZodiacScore'], inplace=True, ascending = (False,False,False,False, False))

display(HTML(show_matching_class.to_html()))

,GNPS_LIBA_superclass,CAN_superclass,GNPS_LIBA_class,CAN_class,GNPS_LIBA_subclass,CAN_subclass,SIR_MF_Zod_ZodiacScore,SIR_MF_Zod_molecularFormula,GNPS_LIB_Adduct,SIR_MF_Zod_adduct,GNPS_LIB_MQScore,GNPS_LIB_MZErrorPPM,GNPS_LIB_SharedPeaks,GNPS_LIB_SpecMZ,Match_GNPSsuperclass-SIRIUS,Match_GNPSclass-SIRIUS,Match_GNPSsubclass-SIRIUS
#featureID,,,,,,,,,,,,,,,,,
6114,NaN,Lipids and lipid-like molecules,NaN,Glycerolipids,NaN,Diradylglycerols,0.946,C35H72O5,M+H,[M + Na]+,0.735448,2.152270,7.0,595.531,yes,yes,yes
3846,Lipids and lipid-like molecules,Lipids and lipid-like molecules,Glycerolipids,Glycerolipids,Diradylglycerols,Diradylglycerols,0.974,C37H70O5,M+H-H2O,[M - H2O + H]+,0.765873,2.642130,12.0,577.521,yes,yes,yes
24697,Lipids and lipid-like molecules,Lipids and lipid-like molecules,Glycerolipids,Glycerolipids,Diradylglycerols,Diradylglycerols,0.999,C35H68O5,M+H-H2O,[M - H2O + H]+,0.863114,1.881400,12.0,551.502,yes,yes,yes
9160,Benzenoids,Benzenoids,Benzene and substituted derivatives,Benzene and substituted derivatives,Benzoic acids and derivatives,Benzoic acids and derivatives,0.863,C26H42O4,M+H,[M + H]+,0.777877,5.021780,7.0,419.314,yes,yes,yes
27030,Lipids and lipid-like molecules,Lipids and lipid-like molecules,Steroids and steroid derivatives,Steroids and steroid derivatives,Hydroxysteroids,Hydroxysteroids,1.000,C19H32O5,M+H,[M + Na]+,0.775606,5.293280,11.0,363.215,yes,yes,yes
316,Lipids and lipid-like molecules,Lipids and lipid-like molecules,Glycerolipids,Glycerolipids,Diradylglycerols,Monoradylglycerols,1.000,C21H40O4,M+H,[M + H]+,0.795997,4.441410,9.0,357.298,yes,yes,yes
33405,Lipids and lipid-like molecules,Lipids and lipid-like molecules,Fatty Acyls,Glycerolipids,Fatty acid esters,Monoradylglycerols,0.993,C19H40O3,M+H-H2O,[M + Na]+,0.864691,6.745920,11.0,339.287,yes,yes,yes
257,Lipids and lipid-like molecules,Lipids and lipid-like molecules,Fatty Acyls,Glycerolipids,Fatty acid esters,Monoradylglycerols,1.000,C19H36O4,M+H,[M + H]+,0.738777,3.892680,10.0,329.267,yes,yes,yes
1173,Lipids and lipid-like molecules,Lipids and lipid-like molecules,Fatty Acyls,Glycerolipids,Lineolic acids and derivatives,Monoradylglycerols,1.000,C17H34O4,M+H,[M + H]+,0.755989,5.534870,10.0,303.251,yes,yes,yes
